In [ ]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average, F2fluxconverter
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 128
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9
max_shift = 96
lmbda = 1e-30
#noise = 1e30
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
#image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
#I_out = 10*np.array([0, 0, 0, 30, 10,10,300])
I_out = 10*np.array([0, 0, 0, 3, 1,1,117])
#I_out = 10*np.array([0, 0, 0, 0, 0,0,122])

LR_list = [0,0,0,2e-2,3e-2,5e-2,1e-1]

/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch, gc

cuda_tensors = []
for obj in gc.get_objects():
    if isinstance(obj, torch.Tensor) and obj.is_cuda:
        cuda_tensors.append(obj)

print(f"Found {len(cuda_tensors)} CUDA tensors:")
for t in cuda_tensors:
    print(f"  • shape={tuple(t.shape)}, dtype={t.dtype}, bytes={t.element_size()*t.numel():,}")

In [ ]:
import torch

# how much your tensors are _actually_ holding
print("allocated:", torch.cuda.memory_allocated()/2**30, "GiB")

# how much the allocator has reserved from the driver
print("reserved: ", torch.cuda.memory_reserved() /2**30, "GiB")

# a detailed breakdown
print(torch.cuda.memory_summary())

In [ ]:
pty = Ptychography(device='cuda')
# ... after loading your ground truth/object and computing itilde ...
o0 = torch.randn_like(image_tensor__, dtype=torch.complex64, device=device)
L0 = linOperator.estimate_lipschitz(o0, image_tensor__)
print(f"Estimated L = {L0:.3e}, suggested lr = {1.0/L0:.3e}")

In [3]:
import torch, gc

cuda_tensors = []
for obj in gc.get_objects():
    if isinstance(obj, torch.Tensor) and obj.is_cuda:
        cuda_tensors.append(obj)

print(f"Found {len(cuda_tensors)} CUDA tensors:")
for t in cuda_tensors:
    print(f"  • shape={tuple(t.shape)}, dtype={t.dtype}, bytes={t.element_size()*t.numel():,}")

Found 10 CUDA tensors:
  • shape=(1, 1, 512, 512), dtype=torch.complex128, bytes=4,194,304
  • shape=(512, 512), dtype=torch.complex64, bytes=2,097,152
  • shape=(512, 512), dtype=torch.complex64, bytes=2,097,152
  • shape=(1, 361, 512, 512), dtype=torch.float64, bytes=757,071,872
  • shape=(1, 1, 2, 1), dtype=torch.float64, bytes=16
  • shape=(1, 1, 1, 2), dtype=torch.float64, bytes=16
  • shape=(1, 361, 512, 512), dtype=torch.complex64, bytes=757,071,872
  • shape=(1, 1, 2, 2), dtype=torch.float64, bytes=32
  • shape=(1, 361, 512, 512), dtype=torch.complex64, bytes=757,071,872
  • shape=(1, 361, 512, 512), dtype=torch.complex64, bytes=757,071,872


/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/.venv/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:181: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [2]:
o = torch.randn_like(image_tensor__, dtype=torch.complex64, device=device)
2.0 * float((linOperator.probe.abs()**2).max()) * float((2*(linOperator.apply_linop(o).abs()**2).clamp(min=1e-8) - image_tensor__).abs().max())


RuntimeError: CUDA out of memory. Tried to allocate 722.00 MiB (GPU 0; 23.66 GiB total capacity; 21.87 GiB already allocated; 663.38 MiB free; 21.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF